このプログラムはvscodeではなく、anacondaから"base"環境でjupyternotebookを起動し、"Slicer5.6"カーネルを選択して実行する
参考：https://www.youtube.com/watch?v=jcRsRw6RC2g&pp=ygUVc2xpY2VyIHB5dGhvbiBqdXB5dGVy

databaseにdicomをimportする処理を作成する。

In [1]:
import slicer
from DICOMLib import DICOMUtils
import os

## DICOMデータをSlicer databaseにimportする処理

In [2]:
dicomFilesDirectory = "C://Users/user/Documents/liver/inputs/images/15680/" # 指定したフォルダ内のdicomのみimport
# dicomFilesDirectory = "C://Users/user/Documents/liver/inputs/images/"  # これで全データをimport

# instantiate a new DICOM browser
slicer.util.selectModule("DICOM")
dicomBrowser = slicer.modules.DICOMWidget.browserWidget.dicomBrowser
# use dicomBrowser.ImportDirectoryCopy to make a copy of the files (useful for importing data from removable storage)
dicomBrowser.importDirectory(dicomFilesDirectory, dicomBrowser.ImportDirectoryAddLink)
# wait for import to finish before proceeding (optional, if removed then import runs in the background)
dicomBrowser.waitForImportFinished()

In [ ]:
# 読み込み結果の確認
patientUIDs = slicer.dicomDatabase.patients()
print(patientUIDs)
print(len(patientUIDs))

## データの読み込みと書き出し

In [71]:
dicomFilesDirectory = "C://Users/user/Documents/liver/inputs/images/"  # これで全データをimportすることができた
# instantiate a new DICOM browser
slicer.util.selectModule("DICOM")
dicomBrowser = slicer.modules.DICOMWidget.browserWidget.dicomBrowser
# use dicomBrowser.ImportDirectoryCopy to make a copy of the files (useful for importing data from removable storage)
dicomBrowser.importDirectory(dicomFilesDirectory, dicomBrowser.ImportDirectoryAddLink)
# wait for import to finish before proceeding (optional, if removed then import runs in the background)
dicomBrowser.waitForImportFinished()

In [ ]:
# slicer.dicomDatabase.imagesCount()
# slicer.dicomDatabase.patientsCount()
patientUIDs = slicer.dicomDatabase.patients()
print(patientUIDs)
print(len(patientUIDs))
for ids in patientUIDs:
    slicer.dicomDatabase.removePatient(ids) # これで指定した患者を消せる！
patientUIDs = slicer.dicomDatabase.patients()
print(patientUIDs)
print(len(patientUIDs))

In [ ]:
# descriptionの出現回数を保存する辞書を作成
from collections import Counter
# Initialize an empty Counter object
string_counter = Counter()
# Update the counter with the strings
def update_counter(string):
    string_counter[string] += 1
# スペースを"_"に置き換える関数
import re
def replace_spaces_to_underscore(string):
    # Replace consecutive spaces with a single underscore
    string = re.sub(r'\s+', '_', string)
    # Replace full-width spaces with an underscore
    string = string.replace('　', '_')
    return string
    
# descriptionの一覧を表示
patientUIDs = slicer.dicomDatabase.patients()
for patientUID in patientUIDs:
    studyList = slicer.dicomDatabase.studiesForPatient(patientUID)
    for study in studyList:
        seriesList = slicer.dicomDatabase.seriesForStudy(study)
        for series in seriesList:
            description = slicer.dicomDatabase.descriptionForSeries(series)
#             print(description)
#             fileList = db.filesForSeries(seriesList[0])
            update_counter(replace_spaces_to_underscore(description))
print(string_counter)

In [ ]:
# 同じファイル名になる場合に対処
from collections import Counter
# Initialize an empty Counter object
check_duplication = Counter()
# Update the counter with the strings
def check_dup(string):
    check_duplication[string] += 1

# input, outputフォルダーの定義
input_path = "C://Users/user/Documents/liver/inputs/images/"  # これで全データをimport
outputFolder = "C://Users/user/Documents/liver/niigz/"

for file_name in os.listdir(input_path): # file_name: 15678,15679,...
    if not os.path.exists(os.path.join(outputFolder,file_name)):
        os.mkdir(os.path.join(outputFolder,file_name)) # 症例番号ごとのフォルダを作成
    
    dicomFilesDirectory = os.path.join(input_path, file_name)

# dicomデータをslicer databaseにimport
    # instantiate a new DICOM browser
    slicer.util.selectModule("DICOM")
    dicomBrowser = slicer.modules.DICOMWidget.browserWidget.dicomBrowser
    # use dicomBrowser.ImportDirectoryCopy to make a copy of the files (useful for importing data from removable storage)
    dicomBrowser.importDirectory(dicomFilesDirectory, dicomBrowser.ImportDirectoryAddLink)
    # wait for import to finish before proceeding (optional, if removed then import runs in the background)
    dicomBrowser.waitForImportFinished()
    
# nii.gzに変換して書き出し
    patientUIDs = slicer.dicomDatabase.patients()
    for patientUID in patientUIDs:
        loadedNodeIDs = DICOMUtils.loadPatientByUID(patientUID)
        for loadedNodeID in loadedNodeIDs:
            # Check if we want to save this node
            node = slicer.mrmlScene.GetNodeByID(loadedNodeID)
            # Only export images
            if not node or not node.IsA('vtkMRMLScalarVolumeNode'):
                continue
            # Construct filename
            shNode = slicer.mrmlScene.GetSubjectHierarchyNode()
            seriesItem = shNode.GetItemByDataNode(node)
            studyItem = shNode.GetItemParent(seriesItem)
            patientItem = shNode.GetItemParent(studyItem)
            filename = shNode.GetItemAttribute(patientItem, 'DICOM.PatientID')
            filename += '_' + shNode.GetItemAttribute(studyItem, 'DICOM.StudyDate')
            filename += '_' + shNode.GetItemAttribute(seriesItem, 'DICOM.SeriesNumber')
            filename += '_' + shNode.GetItemAttribute(seriesItem, 'DICOM.Modality')
            check_dup(filename)#同じ名前のファイルが出力され上書きされるのを防ぐ
            if check_duplication[filename]>1:
                filename += '_' + str(check_duplication[filename]-1)
#             print(filename)
            filename = slicer.app.ioManager().forceFileNameValidCharacters(filename) + ".nii.gz"
            # Save node
            print(f'Write {node.GetName()} to {filename}')
            output_path = os.path.join(outputFolder,file_name)
            output_path = os.path.join(output_path,filename)
            print(output_path)
            success = slicer.util.saveNode(node, output_path)
        slicer.mrmlScene.Clear()
    for ids in patientUIDs:
        slicer.dicomDatabase.removePatient(ids) # これで指定した患者を消せる！